In [57]:
import pandas as pd
import numpy as np
import gc
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook as tqdm
import re
import pickle
from collections import Counter
import json
import operator
from functools import reduce
import glob
%matplotlib inline

In [2]:
#2015-01-02:2017-12-28 weights            2015-05-01：2017-05-01 time
#first 18 includes DJI which should be excluded as it is not treated the same as the rest
#strategy is buy component stock buy C + buy P
#index: sell C + sell P
#'DJI': 102456
#ticker with incomplete data: 211899 210354

In [3]:
ticker_to_secid = {
    'AA': 210354,
    'AAPL': 101594,
    'AXP': 101375,
    'BA': 102265,
    'BAC': 101966,
    'CAT': 102796,
    'CSCO': 103042,
    'CVX': 102968,
    'DIS': 103879,
    'DJI': 102456,
    'DWDP': 211899,
    'GE': 105169,
    'GS': 105329,
    'HD': 105759,
    'HPQ': 105700,
    'IBM': 106276,
    'INTC': 106203,
    'JNJ': 106566,
    'JPM': 102936,
    'KO': 103125,
    'MCD': 107318,
    'MMM': 107616,
    'MRK': 107430,
    'MSFT': 107525,
    'NKE': 108161,
    'PFE': 108948,
    'PG': 109224,
    'T': 109775,
    'TRV': 109869,
    'UNH': 111469,
    'UTX': 111459,
    'V': 135419,
    'VZ': 111668,
    'WBA': 111861,
    'WMT': 111860,
    'XOM': 104533
}

In [19]:
secid_to_ticker = {v:k for k,v in ticker_to_secid.items()}
sec_id_list = list(ticker_to_secid.values())
ticker_list = list(ticker_to_secid.keys())

In [22]:
ticker_list_no_na = [secid_to_ticker[x] for x in sec_id_list_no_na]

In [48]:
ticker_list_no_na;

In [6]:
sec_id_list_no_na = [
    101594, 101375, 102265, 101966, 102796, 103042, 102968, 103879, 105169,
    105329, 105759, 105700, 106276, 106203, 106566, 102936, 103125, 107318,
    107616, 107430, 107525, 108161, 108948, 109224, 109775, 109869, 111469,
    111459, 135419, 111668, 111861, 111860, 104533
]

In [7]:
len(sec_id_list_no_na)

33

In [ ]:
dji_df = pd.read_csv('dji_component.csv', index_col = 0)
# 2015-01-02:2017-12-28

In [8]:
option_dfs_17 = {}
option_dfs_33 = {}

In [ ]:
#this is dow jones index
#sec_id_list[9]

In [9]:
for sec_id in tqdm(sec_id_list_no_na[:17]):
    option_dfs_17[sec_id]= pd.read_csv('./option_data/'+str(sec_id)+'.csv', index_col = 'date', parse_dates =['date', 'exdate'])

In [10]:
%%time
f = open("option_dfs_17.pkl","wb")
pickle.dump(option_dfs_17,f)
f.close()

Wall time: 58.2 s


In [38]:
del option_dfs_17
gc.collect;

In [12]:
for sec_id in tqdm(sec_id_list[17:33]):
    option_dfs_33[sec_id]= pd.read_csv('./option_data/'+str(sec_id)+'.csv', index_col = 'date', parse_dates =['date', 'exdate'])

f = open("option_dfs_33.pkl","wb")
pickle.dump(option_dfs_33,f)
f.close()

In [45]:
del option_dfs_33
gc.collect;

In [26]:
%%time
#preprocess 17 first
with open('option_dfs_17.pkl', 'rb') as handle:
    option_dfs_17 = pickle.load(handle)

Wall time: 14.4 s


In [39]:
%%time
#preprocess 33 later
with open('option_dfs_33.pkl', 'rb') as handle:
    option_dfs_33 = pickle.load(handle)

Wall time: 10.9 s


In [50]:
option_dfs_hpq_wmt_xom_wba = {}
option_dfs_hpq_wmt_xom_wba[ticker_to_secid['HPQ']] = pd.read_csv('./option_data/'+str(ticker_to_secid['HPQ'])+'.csv', index_col = 'date', parse_dates =['date', 'exdate'])
option_dfs_hpq_wmt_xom_wba[ticker_to_secid['WMT']] = pd.read_csv('./option_data/'+str(ticker_to_secid['WMT'])+'.csv', index_col = 'date', parse_dates =['date', 'exdate'])
option_dfs_hpq_wmt_xom_wba[ticker_to_secid['XOM']] = pd.read_csv('./option_data/'+str(ticker_to_secid['XOM'])+'.csv', index_col = 'date', parse_dates =['date', 'exdate'])
option_dfs_hpq_wmt_xom_wba[ticker_to_secid['WBA']] = pd.read_csv('./option_data/'+str(ticker_to_secid['WBA'])+'.csv', index_col = 'date', parse_dates =['date', 'exdate'])



In [ ]:
del option_dfs_hpq_wmt_xom_wba
gc.collect()

In [53]:
def debug():
    #global sec_id_list_no_na, option_dfs_33
    global option_dfs_
    stock_df_all = pd.read_csv('all_stock.csv', index_col = 0, parse_dates = True) 
    stock_df_all = stock_df_all[pd.notnull(stock_df_all['DJI'])]
    print(stock_df_all.columns)
    for sec_id in tqdm([ticker_to_secid['WBA'],100000]): #change 0:1 to :17 whence finishing debug
        
        assert secid_to_ticker[sec_id] == 'WBA'
        print(f'producing result dataframe for {secid_to_ticker[sec_id]}')
        option_dfs_hpq_wmt_xom_wba[sec_id] = option_dfs_hpq_wmt_xom_wba[sec_id]['2015-05-01':'2017-05-01'] #get relevant datetime
        #print('breakpoint_0 here')
        option_df = option_dfs_hpq_wmt_xom_wba[sec_id].copy(deep = True)
        #print('breakpoint_1 here')
        stock_df = pd.DataFrame(stock_df_all[secid_to_ticker[sec_id]]['2015-05-01':'2017-05-01'])
        #print('breakpoint_2 here')
        set_diff = list(set(option_df.index.unique()).difference(stock_df.index.unique()))
        
        #print('breakpoint_3 here')
        if len(option_df.index.unique()) > len(stock_df.index.unique()):
            option_df = option_df.drop(set_diff)
        elif len(option_df.index.unique()) < len(stock_df.index.unique()):
            stock_df = stock_df.drop(set_diff)
        
        #print('stock_df length ', len(stock_df))
        option_df['Adj_Close'] = np.zeros(len(option_df))
        num_list = list(Counter(pd.to_datetime(option_df.index)).values())
        
        #print('length of num_list', len(num_list))
        
        #print('length of enumerate', len(stock_df[secid_to_ticker[sec_id]]))
        big_list = [] #spot price list for the option_df
        
        for i, price in tqdm(enumerate(stock_df[secid_to_ticker[sec_id]].values)):
            big_list.append(([price]*num_list[i]))
        big_list = reduce(operator.concat, big_list)
        
        option_df['Adj_Close'] = big_list
        
        option_df['strike_price_adjusted'] = option_df['strike_price'] / 1000
        option_df['atm_diff'] = np.abs(option_df['strike_price_adjusted'] - option_df['Adj_Close']) 
        option_df['date'] = option_df.index
        #handling 1 month forward contract e.g for days in Sep, we buy contracts that ends in Oct
        option_df['theoretic_end_month'] = option_df['date'].dt.month + 1
        option_df['exdate_flag'] = option_df['theoretic_end_month'] == option_df['exdate'].dt.month
        option_df_exdate_1_month = option_df[option_df['exdate_flag'] == True]
   
        del option_df
        gc.collect();
        #handling at the money put-call pair 
        option_df_exdate_1_month['atm_flag'] = np.full(len(option_df_exdate_1_month), np.inf)
        min_list = []
        
        for date in tqdm(option_df_exdate_1_month['date']):

            min_val = option_df_exdate_1_month.loc[pd.to_datetime(date)]['atm_diff'].min()
            min_list.append(min_val)
        
        option_df_exdate_1_month['atm_flag'] = option_df_exdate_1_month['atm_diff'].isin(min_list).astype(int)
        option_df_exdate_1_month_atm = option_df_exdate_1_month[option_df_exdate_1_month['atm_flag']==1]
        
        #print(option_df_exdate_1_month_atm.head())
        
        del option_df_exdate_1_month
        gc.collect();
        #aggregate by sum to that the vega and theta would directly be the vega and theta of the straddle
        straddle_df = option_df_exdate_1_month_atm.groupby('date').agg('sum')
        
        #print(straddle_df.head(6))
        
        del option_df_exdate_1_month_atm
        gc.collect();
        
        #compute average of implied volatility
        straddle_df['impl_volatility'] = straddle_df['impl_volatility'].div(straddle_df['atm_flag'], axis = 0)
        straddle_df['strike_price_adjusted'] = straddle_df['strike_price_adjusted'].div(straddle_df['atm_flag'], axis = 0)
        straddle_df['Adj_Close'] = straddle_df['Adj_Close'].div(straddle_df['atm_flag'], axis = 0)
        straddle_df['vega'] = straddle_df['vega'].div(straddle_df['atm_flag']/ 2.0, axis = 0)
        straddle_df['theta'] = straddle_df['theta'].div(straddle_df['atm_flag']/ 2.0, axis = 0)
        
        straddle_df_output = straddle_df[['Adj_Close', 'vega', 'theta', 'strike_price_adjusted', 'impl_volatility']]
        
        del straddle_df
        gc.collect();
        
        straddle_df_output.columns = straddle_df_output.columns.map(lambda x: str(x) + '_' + str(secid_to_ticker[sec_id]))
        straddle_df_output.to_csv('./Outputs/'+str(secid_to_ticker[sec_id])+'.csv')
        
        del straddle_df_output
        gc.collect();
        
    return None


In [54]:
debug()

Index(['AA', 'AAPL', 'AXP', 'BA', 'BAC', 'CAT', 'CSCO', 'CVX', 'DIS', 'DJI',
       'GE', 'GS', 'HD', 'HP', 'IBM', 'INTC', 'JNJ', 'JPM', 'KO', 'MCD', 'MMM',
       'MRK', 'MSFT', 'NKE', 'PFE', 'PG', 'T', 'TRV', 'UNH', 'UTX', 'V', 'VZ',
       'WBA', 'WMT', 'XOM'],
      dtype='object')


producing result dataframe for WBA


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:67: FutureWarning: 'date' is both an index level and a column label.
Defaulting to column, but this will raise an ambiguity error in a future version


KeyError: 100000

In [14]:
option_df_dji = {}
option_df_dji[102456] = pd.read_csv('./option_data/'+str(102456)+'.csv', index_col = 'date', parse_dates =['date', 'exdate'])

In [17]:
#for dow jones

def dji_debug():
    global option_dfs_dji
    stock_df_all = pd.read_csv('all_stock.csv', index_col = 0, parse_dates = True) 
    #stock_df_all = stock_df_all[pd.notnull(stock_df_all['DJI'])]
    for sec_id in tqdm([ticker_to_secid['DJI'], 100000]): #100000 is made up stuff to keep the structure of a loop
        
        print(f'producing result dataframe for {secid_to_ticker[sec_id]}')
        assert secid_to_ticker[sec_id] == 'DJI'
        
        option_df_dji[sec_id] = option_df_dji[sec_id]['2015-05-01':'2017-05-01'] #get relevant datetime
        #print('breakpoint_0 here')
        option_df = option_df_dji[sec_id].copy(deep = True)
        #print('breakpoint_1 here')
        stock_df = pd.DataFrame(stock_df_all[secid_to_ticker[sec_id]]['2015-05-01':'2017-05-01'])
        #print('breakpoint_2 here')
        set_diff = list(set(option_df.index.unique()).difference(stock_df.index.unique()))
        
        #print('breakpoint_3 here')
        if len(option_df.index.unique()) > len(stock_df.index.unique()):
            option_df = option_df.drop(set_diff)
        elif len(option_df.index.unique()) < len(stock_df.index.unique()):
            stock_df = stock_df.drop(set_diff)
        
        #print('stock_df length ', len(stock_df))
        option_df['Adj_Close'] = np.zeros(len(option_df))
        num_list = list(Counter(pd.to_datetime(option_df.index)).values())
        
        #print('length of num_list', len(num_list))
        
        #print('length of enumerate', len(stock_df[secid_to_ticker[sec_id]]))
        big_list = [] #spot price list for the option_df
        
        for i, price in tqdm(enumerate(stock_df[secid_to_ticker[sec_id]].values)):
            big_list.append(([price]*num_list[i]))
        big_list = reduce(operator.concat, big_list)
        
        option_df['Adj_Close'] = big_list
        
        option_df['strike_price_adjusted'] = option_df['strike_price'] / 10 #note that DJI multiplier is 10
        option_df['atm_diff'] = np.abs(option_df['strike_price_adjusted'] - option_df['Adj_Close']) 
        option_df['date'] = option_df.index
        #handling 1 month forward contract e.g for days in Sep, we buy contracts that ends in Oct
        option_df['theoretic_end_month'] = option_df['date'].dt.month + 1
        option_df['exdate_flag'] = option_df['theoretic_end_month'] == option_df['exdate'].dt.month
        option_df_exdate_1_month = option_df[option_df['exdate_flag'] == True]
   
        del option_df
        gc.collect();
        #handling at the money put-call pair 
        option_df_exdate_1_month['atm_flag'] = np.full(len(option_df_exdate_1_month), np.inf)
        min_list = []
        
        for date in tqdm(option_df_exdate_1_month['date']):

            min_val = option_df_exdate_1_month.loc[pd.to_datetime(date)]['atm_diff'].min()
            min_list.append(min_val)
        
        option_df_exdate_1_month['atm_flag'] = option_df_exdate_1_month['atm_diff'].isin(min_list).astype(int)
        option_df_exdate_1_month_atm = option_df_exdate_1_month[option_df_exdate_1_month['atm_flag']==1]
        
        #print(option_df_exdate_1_month_atm.head())
        
        del option_df_exdate_1_month
        gc.collect();
        #aggregate by sum to that the vega and theta would directly be the vega and theta of the straddle
        straddle_df = option_df_exdate_1_month_atm.groupby('date').agg('sum')
        
        #print(straddle_df.head(6))
        
        del option_df_exdate_1_month_atm
        gc.collect();
        
        #compute average of implied volatility
        straddle_df['impl_volatility'] = straddle_df['impl_volatility'].div(straddle_df['atm_flag'], axis = 0)
        straddle_df['strike_price_adjusted'] = straddle_df['strike_price_adjusted'].div(straddle_df['atm_flag'], axis = 0)
        straddle_df['Adj_Close'] = straddle_df['Adj_Close'].div(straddle_df['atm_flag'], axis = 0)
        straddle_df['vega'] = straddle_df['vega'].div(straddle_df['atm_flag']/ 2.0, axis = 0)
        straddle_df['theta'] = straddle_df['theta'].div(straddle_df['atm_flag']/ 2.0, axis = 0)
        
        straddle_df_output = straddle_df[['Adj_Close', 'vega', 'theta', 'strike_price_adjusted', 'impl_volatility']]
        
        del straddle_df
        gc.collect();
        
        straddle_df_output.columns = straddle_df_output.columns.map(lambda x: str(x) + '_' + str(secid_to_ticker[sec_id]))
        straddle_df_output.to_csv('./Outputs/'+str(secid_to_ticker[sec_id])+'.csv')
        
        del straddle_df_output
        gc.collect();
        
    return None


In [18]:
dji_debug()

producing result dataframe for DJI


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:68: FutureWarning: 'date' is both an index level and a column label.
Defaulting to column, but this will raise an ambiguity error in a future version


KeyError: 100000

In [64]:
path = r'./Outputs' # use your path
all_files = glob.glob(path + "/*.csv")

In [68]:
all_files.remove('./Outputs\\AAPL.csv')

In [83]:
first_df = pd.read_csv('./Outputs\\AAPL.csv', index_col = 0, parse_dates = True)

In [81]:
del first_df
gc.collect();

In [84]:
for filename in tqdm(all_files):
    tmp_df = pd.read_csv(filename, index_col=0, parse_dates = True)
    first_df = first_df.join(tmp_df)
    #df1.join(df2)
    del tmp_df
    gc.collect();

In [85]:
first_df.head()

,Adj_Close_AAPL,vega_AAPL,theta_AAPL,strike_price_adjusted_AAPL,impl_volatility_AAPL,Adj_Close_AXP,vega_AXP,theta_AXP,strike_price_adjusted_AXP,impl_volatility_AXP,...,Adj_Close_WMT,vega_WMT,theta_WMT,strike_price_adjusted_WMT,impl_volatility_WMT,Adj_Close_XOM,vega_XOM,theta_XOM,strike_price_adjusted_XOM,impl_volatility_XOM
date,,,,,,,,,,,,,,,,,,,,,
2015-05-01,120.220688,24.592235,-31.288457,120.0,0.266121,72.806160,11.209653,-9.831198,73.0,0.182940,...,70.807358,4.031378,-4.258377,71.0,0.111720,76.477814,2.420475,-2.910611,76.5,0.127041
2015-05-04,119.987633,23.288624,-30.393473,120.0,0.255084,73.368507,9.875354,-9.429772,73.5,0.180267,...,71.329834,3.352818,-3.713743,71.5,0.107899,76.701599,2.064004,-2.738732,76.5,0.129785
2015-05-05,117.283951,20.631163,-32.479980,117.0,0.270121,72.956108,11.103545,-12.736984,73.0,0.211315,...,70.383942,6.442647,-16.742333,70.0,0.263768,76.271233,4.786442,-14.671919,76.5,0.257931
2015-05-06,116.547424,22.435245,-35.689655,117.0,0.285550,72.965508,11.273447,-13.992166,73.0,0.222117,...,70.393013,5.288666,-5.917536,70.0,0.186177,75.909729,6.226282,-20.782588,76.5,0.342186
2015-05-07,117.268303,22.650308,-29.379110,117.0,0.267944,73.509102,9.340515,-7.915075,73.5,0.168798,...,70.737488,5.861515,-5.920061,70.5,0.179644,75.401871,5.181361,-12.996575,75.0,0.315902


In [86]:
first_df.to_csv('combined_data.csv')

In [ ]:
shabi_df =pd.DataFrame(np.array([[1, 2, 3,4, 5], [4, 5, 6,7,8], [7, 8, 9, 10, 11], [10, 11, 12,13, 14], [14,15,16,17, 18]]),
...                    columns=['vega1', 'weight1', 'vega2', 'weight2', 'index_vega'])
shabi_df.head()

In [ ]:
shabi_df['vega_components'] = shabi_df['vega1'] * shabi_df['weight1'] + shabi_df['vega2'] * shabi_df['weight2']

In [ ]:
shabi_df['multiplier'] = shabi_df['index_vega'] / shabi_df['vega_components']

In [ ]:
shabi_df.head()

In [ ]:
#vega neutral

sum = 0.0

for i in range(0, 4, 2):
    tmp = shabi_df.iloc[:,i] * shabi_df.iloc[:, i+1]
    sum += tmp
    
sum